# CrateDB Cloud Import

This is an example notebook demonstrating how to load data from
files using the [Import API] interface of [CrateDB Cloud] into
a [CrateDB Cloud Cluster].

The supported file types are CSV, JSON, Parquet, optionally with
gzip compression. They can be acquired from the local filesystem,
or from remote HTTP and AWS S3 resources.

[CrateDB Cloud]: https://cratedb.com/docs/cloud/
[CrateDB Cloud Cluster]: https://cratedb.com/docs/cloud/en/latest/reference/services.html
[Import API]: https://community.cratedb.com/t/importing-data-to-cratedb-cloud-clusters/1467

## Setup

To install the client SDK, use `pip`.

In [ ]:
#!pip install 'cratedb-toolkit'

## Configuration

The notebook assumes you are appropriately authenticated to the CrateDB Cloud
platform, for example using `croud login --idp azuread`. To inspect the list
of available clusters, run `croud clusters list`.

For addressing a database cluster, and obtaining corresponding credentials,
the program uses environment variables, which you can define interactively,
or store them within a `.env` file.

You can use those configuration snippet as a blueprint. Please adjust the
individual settings accordingly.
```shell
CRATEDB_CLOUD_CLUSTER_NAME=Hotzenplotz
CRATEDB_USERNAME='admin'
CRATEDB_PASSWORD='H3IgNXNvQBJM3CiElOiVHuSp6CjXMCiQYhB4I9dLccVHGvvvitPSYr1vTpt4'
```

## Acquire Database Cluster

As a first measure, acquire a resource handle, which manages a CrateDB Cloud
cluster instance.

For effortless configuration, it will obtain configuration settings from
environment variables as defined above.

In [6]:
from cratedb_toolkit import ManagedCluster, InputOutputResource

cluster = ManagedCluster.from_env().start()

## Import Data

From the [NAB Data Corpus], import the "realKnownCause" dataset. The dataset includes
temperature sensor data of an internal component of an industrial machine, with known
anomaly causes. The first anomaly is a planned shutdown of the machine. The second
anomaly is difficult to detect and directly led to the third anomaly, a catastrophic
failure of the machine.

On this topic, we also recommend the notebook about [MLflow and CrateDB], where the
same dataset is used for time series anomaly detection and forecasting.

[NAB Data Corpus]: https://github.com/numenta/NAB/tree/master/data
[MLflow and CrateDB]: https://github.com/crate/cratedb-examples/tree/main/topic/machine-learning/mlops-mlflow

In [4]:
# Define data source.
url = "https://github.com/crate/cratedb-datasets/raw/main/machine-learning/timeseries/nab-machine-failure.csv"
source = InputOutputResource(url=url)

# Invoke import job. Without `target` argument, the destination
# table name will be derived from the input file name.
cluster.load_table(source=source)

==> Info: Status: REGISTERED (Your import job was received and is pending processing.)
==> Info: Done importing 22.70K records
==> Success: Operation completed.


CloudJob(info={'cluster_id': '09be10b6-7d78-497b-842e-fbb47642d398', 'compression': 'none', 'dc': {'created': '2023-11-17T18:54:04.070000+00:00', 'modified': '2023-11-17T18:54:04.070000+00:00'}, 'destination': {'create_table': True, 'table': 'nab-machine-failure'}, 'file': None, 'format': 'csv', 'id': '56051dc3-ee8e-4a38-9066-73bcd427d05a', 'progress': {'bytes': 0, 'details': {'create_table_sql': None}, 'failed_files': 0, 'failed_records': 0, 'message': 'Import succeeded', 'percent': 100.0, 'processed_files': 1, 'records': 22695, 'total_files': 1, 'total_records': 22695}, 'schema': {'type': 'csv'}, 'status': 'SUCCEEDED', 'type': 'url', 'url': {'url': 'https://github.com/crate/cratedb-datasets/raw/main/machine-learning/timeseries/nab-machine-failure.csv'}}, found=True, _custom_status=None, _custom_message=None)

## Query Data

In order to inspect if the dataset has been imported successfully, run an SQL
command sampling a few records.

In [5]:
# Query data.
cluster.query('SELECT * FROM "nab-machine-failure" LIMIT 5;')

[{'timestamp': 1386021900000, 'value': 80.78327674},
 {'timestamp': 1386024000000, 'value': 81.37357535},
 {'timestamp': 1386024600000, 'value': 80.18124978},
 {'timestamp': 1386030300000, 'value': 82.88189183},
 {'timestamp': 1386030600000, 'value': 83.57965349}]